UNSW-NB15 数据集 BT-TWD 与基线实验

本 notebook 按步骤运行：环境准备 → 数据加载 → 特征构建 → 单次留出与 K 折实验 → 输出检查 → 可选扩展。

本 notebook 结构与 03_bank_bttwd_main.ipynb 保持一致，仅更换 UNSW 配置，并适配 YAML 中 train/test 双文件的读取。

In [1]:
# 步骤0：环境与路径设置
import os, sys
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 将项目根目录加入路径，便于导入 bttwdlib
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root_path not in sys.path:
    sys.path.append(root_path)
# 关键：把 root_path 转成 Path，后面就可以用 "/" 了
root_path = Path(root_path)

from bttwdlib import (
    load_yaml_cfg,
    show_cfg,
    load_dataset,
    prepare_features_and_labels,
    BucketTree,
    run_holdout_experiment,
    run_kfold_experiments,
    log_info,
    set_global_seed,
)

cfg_path = os.path.join(root_path, 'configs', 'unsw_nb15.yaml')
cfg = load_yaml_cfg(cfg_path)
set_global_seed(cfg.get('SEED', {}).get('global_seed', 42))
log_info('【步骤0摘要】环境准备完毕，路径与随机种子已设置。')



【INFO】【2025-11-30 20:27:39】【配置加载】已读取 e:\yan\组\三支决策\机器学习\BT_TWD\configs\unsw_nb15.yaml
【INFO】【2025-11-30 20:27:46】【步骤0摘要】环境准备完毕，路径与随机种子已设置。


In [2]:
# 步骤1：加载 UNSW 原始数据，并按 split 列拆分 train / test
log_info("【步骤1】开始加载 UNSW 原始数据并拆分为训练集 / 测试集……")

df_raw, target_col = load_dataset(cfg)

split_col = "split" if "split" in df_raw.columns else None
if split_col is None:
    raise RuntimeError("UNSW 配置要求显式 train/test 划分（需要 split 列）")

df_train = df_raw[df_raw[split_col].str.lower() == "train"].reset_index(drop=True)
df_test  = df_raw[df_raw[split_col].str.lower() == "test"].reset_index(drop=True)

log_info(f"【步骤1】训练集大小：{len(df_train)}，测试集大小：{len(df_test)}")
pos_train = df_train[target_col].mean()
pos_test  = df_test[target_col].mean()
log_info(f"【步骤1】训练集正类占比：{pos_train:.2%}，测试集正类占比：{pos_test:.2%}")

display(df_train.head())
display(df_train[target_col].value_counts())
display(df_train[target_col].value_counts(normalize=True))

log_info("【步骤1摘要】UNSW 原始数据加载与 train/test 拆分完成，已输出训练集标签分布。")


【INFO】【2025-11-30 20:27:46】【步骤1】开始加载 UNSW 原始数据并拆分为训练集 / 测试集……
【INFO】【2025-11-30 20:27:47】【数据加载】文本表格 ..\data\UNSW-NB15\UNSW_NB15_training-set.csv 已读取，样本数=82332，列数=45
【INFO】【2025-11-30 20:27:48】【数据加载】文本表格 ..\data\UNSW-NB15\UNSW_NB15_testing-set.csv 已读取，样本数=175341，列数=45
【INFO】【2025-11-30 20:27:48】【数据加载】检测到显式 train/test 配置，训练集 n=82332，测试集 n=175341
【INFO】【2025-11-30 20:27:49】【数据集信息】名称=UNSW_NB15，样本数=257673，目标列=label，正类比例=63.91%；训练集正类比例=55.06%；测试集正类比例=68.06%
【INFO】【2025-11-30 20:27:49】【步骤1】训练集大小：82332，测试集大小：175341
【INFO】【2025-11-30 20:27:49】【步骤1】训练集正类占比：55.06%，测试集正类占比：68.06%


,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label,split
0,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,...,1,2,0,0,0,1,2,0,0,train
1,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,...,1,2,0,0,0,1,2,0,0,train
2,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,...,1,3,0,0,0,1,3,0,0,train
3,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,...,1,3,0,0,0,2,3,0,0,train
4,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,...,1,3,0,0,0,2,3,0,0,train


label
1    45332
0    37000
Name: count, dtype: int64

label
1    0.5506
0    0.4494
Name: proportion, dtype: float64

【INFO】【2025-11-30 20:27:49】【步骤1摘要】UNSW 原始数据加载与 train/test 拆分完成，已输出训练集标签分布。


In [3]:
# 步骤2：特征构建 & 拆分 X / y / 桶特征
log_info("【步骤2】开始根据 YAML 配置构建特征，并拆分为 X / y / 桶特征……")

# 注意：UNSW 的 prepare_features_and_labels 返回第三个是 meta 字典，不是 DataFrame
X_train, y_train, meta = prepare_features_and_labels(df_train, cfg)

# 按原来的逻辑，从 df_train 中挑出用于分桶的特征列
prep_cfg = cfg.get("PREPROCESS", {})
bucket_cols = (prep_cfg.get("continuous_cols") or []) + (prep_cfg.get("categorical_cols") or [])
bucket_df_train = df_train[bucket_cols].reset_index(drop=True)

log_info(
    f"【步骤2】特征构建完成，X_train 形状={X_train.shape}，"
    f"y_train 长度={len(y_train)}，bucket_df_train 形状={bucket_df_train.shape}。"
)
display(bucket_df_train.head())

log_info("【步骤2摘要】UNSW 训练集特征构建完成，后续将基于该特征进行 BT-TWD 与基线实验。")


【INFO】【2025-11-30 20:27:49】【步骤2】开始根据 YAML 配置构建特征，并拆分为 X / y / 桶特征……
【INFO】【2025-11-30 20:27:49】【预处理】连续特征=0个，类别特征=3个
【INFO】【2025-11-30 20:27:49】【预处理】编码后维度=151
【INFO】【2025-11-30 20:27:49】【步骤2】特征构建完成，X_train 形状=(82332, 151)，y_train 长度=82332，bucket_df_train 形状=(82332, 3)。


,proto,service,state
0,udp,-,INT
1,udp,-,INT
2,udp,-,INT
3,udp,-,INT
4,udp,-,INT


【INFO】【2025-11-30 20:27:49】【步骤2摘要】UNSW 训练集特征构建完成，后续将基于该特征进行 BT-TWD 与基线实验。


In [4]:
# 步骤3：单次留出验证（BT-TWD + 各基线）
log_info("【步骤3】开始执行单次留出验证实验（BT-TWD + 基线）……")

holdout_metrics = run_holdout_experiment(
    X_train,
    y_train,
    bucket_df_train,
    cfg,
)
holdout_df = pd.DataFrame(holdout_metrics)
display(holdout_df)

log_info("【步骤3摘要】单次留出验证完成，已输出各模型的指标表。")


【INFO】【2025-11-30 20:27:49】【步骤3】开始执行单次留出验证实验（BT-TWD + 基线）……
【INFO】【2025-11-30 20:27:50】【数据切分】训练/验证/测试样本数 = 57632/8233/16467，训练正类占比=55.06%
【INFO】【2025-11-30 20:27:50】[BT] 使用桶评分配置：mode=bac_regret, bac_weight=1.0, regret_weight=1.0, regret_sign=-1.0


C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [20:27:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-11-30 20:27:53】【BTTWD】全局模型训练完成，用于兜底预测
[INFO][BT][2025-11-30 20:27:53] 创建桶 bucket_id=ROOT，level=0，parent_id=ROOT，split_name=ROOT，split_type=ROOT，split_rule="all"，n_samples=57632
[INFO][BT][2025-11-30 20:27:53] 创建桶 bucket_id=L1_proto=any，level=1，parent_id=ROOT，split_name=L1_proto，split_type=categorical_group，split_rule="any"，n_samples=68
[INFO][BT][2025-11-30 20:27:53] 创建桶 bucket_id=L1_proto=gre，level=1，parent_id=ROOT，split_name=L1_proto，split_type=categorical_group，split_rule="gre"，n_samples=63
[INFO][BT][2025-11-30 20:27:53] 创建桶 bucket_id=L1_proto=ospf，level=1，parent_id=ROOT，split_name=L1_proto，split_type=categorical_group，split_rule="ospf"，n_samples=476
[INFO][BT][2025-11-30 20:27:53] 创建桶 bucket_id=L1_proto=sctp，level=1，parent_id=ROOT，split_name=L1_proto，split_type=categorical_group，split_rule="sctp"，n_samples=233
[INFO][BT][2025-11-30 20:27:53] 创建桶 bucket_id=L1_proto=tcp，level=1，parent_id=ROOT，split_name=L1_proto，split_type=categorical_group，split_rule="tcp"，n_samples=302

,metrics_s3,metrics_binary
Precision,0.624810,0.624810
Recall,0.996030,0.996030
F1,0.767910,0.767910
BAC,0.631596,0.631596
AUC,0.885099,0.885099
MCC,0.399672,0.399672
Kappa,0.282068,0.282068
BND_ratio,0.000000,0.000000
POS_Coverage,0.877756,NaN
Regret,0.340256,0.340256


【INFO】【2025-11-30 20:28:00】【步骤3摘要】单次留出验证完成，已输出各模型的指标表。


In [5]:
# 步骤4：K 折交叉验证（BT-TWD + 各基线）
log_info("【步骤4】开始执行 K 折交叉验证实验（BT-TWD + 基线）……")

results = run_kfold_experiments(
    X_train,
    y_train,
    bucket_df_train,
    cfg,
)
metrics_df = pd.DataFrame(results)
display(metrics_df)

log_info("【步骤4摘要】K 折交叉验证完成，已展示每个模型在各折上的指标。")


【INFO】【2025-11-30 20:28:00】【步骤4】开始执行 K 折交叉验证实验（BT-TWD + 基线）……
【INFO】【2025-11-30 20:28:00】【基线-LogReg】使用决策阈值=0.200（fixed 模式）


e:\yan\组\三支决策\机器学习\BT_TWD\bttwdlib\baselines.py:103: RuntimeWarning: Mean of empty slice
  summary[f"{col}_mean"] = float(np.nanmean(arr))
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


【INFO】【2025-11-30 20:28:40】【基线-LogReg】整体指标：AUC_mean=0.881, AUC_std=0.003, BAC_mean=0.625, BAC_std=0.003, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.764, F1_std=0.001, Kappa_mean=0.268, Kappa_std=0.006, MCC_mean=0.385, MCC_std=0.005, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.621, Precision_std=0.002, Recall_mean=0.995, Recall_std=0.001, Regret_mean=0.350, Regret_std=0.002
【INFO】【2025-11-30 20:28:40】【基线-RF】使用决策阈值=0.200（fixed 模式）


e:\yan\组\三支决策\机器学习\BT_TWD\bttwdlib\baselines.py:103: RuntimeWarning: Mean of empty slice
  summary[f"{col}_mean"] = float(np.nanmean(arr))
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


【INFO】【2025-11-30 20:30:08】【基线-RF】整体指标：AUC_mean=0.882, AUC_std=0.003, BAC_mean=0.625, BAC_std=0.003, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.765, F1_std=0.001, Kappa_mean=0.269, Kappa_std=0.006, MCC_mean=0.388, MCC_std=0.005, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.621, Precision_std=0.002, Recall_mean=0.996, Recall_std=0.001, Regret_mean=0.347, Regret_std=0.003
【INFO】【2025-11-30 20:30:08】【基线-KNN】使用决策阈值=0.200（fixed 模式）


...

【INFO】【2025-11-30 20:31:16】【BTTWD】全局模型训练完成，用于兜底预测
[INFO][BT][2025-11-30 20:31:17] 创建桶 bucket_id=ROOT，level=0，parent_id=ROOT，split_name=ROOT，split_type=ROOT，split_rule="all"，n_samples=65866
[INFO][BT][2025-11-30 20:31:17] 创建桶 bucket_id=L1_proto=any，level=1，parent_id=ROOT，split_name=L1_proto，split_type=categorical_group，split_rule="any"，n_samples=68
[INFO][BT][2025-11-30 20:31:17] 创建桶 bucket_id=L1_proto=gre，level=1，parent_id=ROOT，split_name=L1_proto，split_type=categorical_group，split_rule="gre"，n_samples=73
[INFO][BT][2025-11-30 20:31:17] 创建桶 bucket_id=L1_proto=ospf，level=1，parent_id=ROOT，split_name=L1_proto，split_type=categorical_group，split_rule="ospf"，n_samples=535
[INFO][BT][2025-11-30 20:31:17] 创建桶 bucket_id=L1_proto=rsvp，level=1，parent_id=ROOT，split_name=L1_proto，split_type=categorical_group，split_rule="rsvp"，n_samples=52
[INFO][BT][2025-11-30 20:31:17] 创建桶 bucket_id=L1_proto=sctp，level=1，parent_id=ROOT，split_name=L1_proto，split_type=categorical_group，split_rule="sctp"，n_samples=25

C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [20:31:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-11-30 20:31:27】【BTTWD】全局模型训练完成，用于兜底预测
[INFO][BT][2025-11-30 20:31:28] 创建桶 bucket_id=ROOT，level=0，parent_id=ROOT，split_name=ROOT，split_type=ROOT，split_rule="all"，n_samples=65866
[INFO][BT][2025-11-30 20:31:28] 创建桶 bucket_id=L1_proto=any，level=1，parent_id=ROOT，split_name=L1_proto，split_type=categorical_group，split_rule="any"，n_samples=83
[INFO][BT][2025-11-30 20:31:28] 创建桶 bucket_id=L1_proto=gre，level=1，parent_id=ROOT，split_name=L1_proto，split_type=categorical_group，split_rule="gre"，n_samples=69
[INFO][BT][2025-11-30 20:31:28] 创建桶 bucket_id=L1_proto=ipv6，level=1，parent_id=ROOT，split_name=L1_proto，split_type=categorical_group，split_rule="ipv6"，n_samples=51
[INFO][BT][2025-11-30 20:31:28] 创建桶 bucket_id=L1_proto=ospf，level=1，parent_id=ROOT，split_name=L1_proto，split_type=categorical_group，split_rule="ospf"，n_samples=541
[INFO][BT][2025-11-30 20:31:28] 创建桶 bucket_id=L1_proto=sctp，level=1，parent_id=ROOT，split_name=L1_proto，split_type=categorical_group，split_rule="sctp"，n_samples=25

,baselines,bttwd
LogReg,"{'per_fold': [{'Precision': 0.618750428640011,...",NaN
RandomForest,{'per_fold': [{'Precision': 0.6190867955573838...,NaN
KNN,{'per_fold': [{'Precision': 0.7263592130315211...,NaN
XGBoost,{'per_fold': [{'Precision': 0.6190867955573838...,NaN
per_fold,NaN,"[{'fold': 1, 'model': 'BTTWD', 'Precision': 0...."
summary,NaN,"[{'model': 'BTTWD', 'Precision_mean': 0.620740..."


【INFO】【2025-11-30 20:31:36】【步骤4摘要】K 折交叉验证完成，已展示每个模型在各折上的指标。


In [6]:
# 步骤5：检查输出目录（结果 / 图像）
log_info("【步骤5】检查输出目录，确认结果文件是否已保存……")

output_cfg = cfg.get("OUTPUT", {})
results_dir = root_path / output_cfg.get("results_dir", "results")
figs_dir = root_path / output_cfg.get("figs_dir", "results/figs")

print("结果目录:", results_dir)
print("图像目录:", figs_dir)

if results_dir.exists():
    print("结果文件：", os.listdir(results_dir))
else:
    print("结果目录不存在。")

if figs_dir.exists():
    print("图像文件：", os.listdir(figs_dir))
else:
    print("图像目录不存在。")

log_info("【步骤5摘要】输出目录检查完成，请根据需要查看 CSV/图像文件。")


【INFO】【2025-11-30 20:31:36】【步骤5】检查输出目录，确认结果文件是否已保存……
结果目录: e:\yan\组\三支决策\机器学习\BT_TWD\results\unsw_nb15
图像目录: e:\yan\组\三支决策\机器学习\BT_TWD\results\unsw_nb15\figs
结果文件： ['bucket_fallback_stats.csv', 'bucket_metrics.csv', 'bucket_metrics_gain.csv', 'bucket_thresholds.csv', 'bucket_thresholds_per_fold.csv', 'bucket_tree_structure.csv', 'metrics_kfold_per_fold.csv', 'metrics_kfold_summary.csv', 'metrics_overview.csv']
图像目录不存在。
【INFO】【2025-11-30 20:31:36】【步骤5摘要】输出目录检查完成，请根据需要查看 CSV/图像文件。


In [7]:
# 步骤6：基于 K 折结果的整体汇总（可选）
log_info("【步骤6】基于 K 折结果做整体指标汇总……")

if 'metrics_df' in locals() and isinstance(metrics_df, pd.DataFrame) and not metrics_df.empty:
    if 'model' in metrics_df.columns:
        aggregated_df = metrics_df.groupby('model').mean(numeric_only=True).reset_index()
        display(aggregated_df)
    else:
        log_info("【步骤6】metrics_df 中未找到 'model' 列，跳过聚合。")
else:
    log_info("【步骤6】暂无可汇总的交叉验证结果，跳过。")

log_info("【步骤6摘要】整体指标汇总完成（若有可用数据）。")


【INFO】【2025-11-30 20:31:36】【步骤6】基于 K 折结果做整体指标汇总……
【INFO】【2025-11-30 20:31:36】【步骤6】metrics_df 中未找到 'model' 列，跳过聚合。
【INFO】【2025-11-30 20:31:36】【步骤6摘要】整体指标汇总完成（若有可用数据）。


In [8]:
# 步骤7：XGB + TWD 全局阈值搜索（UNSW 专用，可选）
try:
    from bttwdlib import run_unsw_xgb_twd_experiment  # type: ignore
    has_unsw_xgb_twd = True
except ImportError:
    has_unsw_xgb_twd = False

if has_unsw_xgb_twd:
    log_info("【步骤7】开始运行 XGB + TWD 基线实验（含全局阈值搜索）……")
    best_alpha, best_beta, best_stats = run_unsw_xgb_twd_experiment(cfg)
    log_info(
        f"【步骤7摘要】XGB + TWD 全局阈值搜索完成："
        f"alpha={best_alpha:.3f}, beta={best_beta:.3f}, "
        f"Regret={best_stats.get('Regret', best_stats.get('regret', float('nan'))):.4f}"
    )
else:
    log_info("【步骤7摘要】未检测到 run_unsw_xgb_twd_experiment 函数，跳过该实验。")


【INFO】【2025-11-30 20:31:36】【步骤7摘要】未检测到 run_unsw_xgb_twd_experiment 函数，跳过该实验。
